In [1]:
from scipy.interpolate import interp1d

In [2]:
d = pd.read_pickle('A123_OCV_vs_SOC_data.pkl')
d = dict(sorted(d.items()))

In [3]:
d.keys()

dict_keys(['N05', 'N15', 'N25', 'P05', 'P15', 'P25', 'P35', 'P45'])

## Model Functions

__Nested Key Finding Function__

In [4]:
def nest_key(d, indent=0):
    
    for key, value in d.items():
        
        print('\t' * indent + f'{key}')
        
        if isinstance(value, dict):
            
            nest_key(value, indent+1)

__Interpolation function__

In [5]:
def interp_ocv(z,OCV):
    
    dum = interp1d(z,OCV)
    
    return dum

## Model Calculations

__OCV vs Temperature model__

We are making assumption that OCV varies linearly with the temperature. 0°C is considered as reference tempearture for fittimg the model. Since we have reference point at 0°C, we are going to remove the negative temperatures from the model fit. Author used ___Linear Least Square Method___ to ge the OCV relation.

$$OCV(Z(t),T(t)) = OCV_{0}[Z(t)] + T(t)\times OCV_{rel}[Z(t)]$$<br>

$$\begin{bmatrix}OCV|T_{1}\\OCV|T_{2}\\ ...\\ OCV|T_{3}\end{bmatrix} =\begin{bmatrix}1 & T_{1}\\ 1 &  T_{2}\\...& ... \\ 1 & T_{3}\end{bmatrix}\cdot\begin{bmatrix}OCV_{0}[Z(t)]\\OCV_{rel}[Z(t))] \end{bmatrix}$$<br>

$$Y = A\cdot X$$

$$X = A\Y$$

In [6]:
temp_list = [5,15,25,35,45]
SOC = d['P25']['SOC']
OCV_0 = np.zeros(len(SOC))
OCV_rel =np.zeros(len(SOC))

In [7]:
A = np.ones([len(temp_list),2])
A[:,1] = temp_list
Y = np.zeros([len(temp_list),len(SOC)])

In [8]:
for i in np.arange(0,len(temp_list),1):
    
    dum = f'P{str(temp_list[i]).zfill(2)}'
    
    Y[i] = d[dum]['ocv']

In [9]:
for i in range(len(SOC)):
    
    X = np.linalg.lstsq(A,Y[:,i], rcond = None)
    
    OCV_0[i] = X[0][0]
    OCV_rel[i] = X[0][1]

We are more interested in getting the SOC prediction if the voltage is known to us. Thus we will give voltage as input to get the SOC relation with temperature.

$$SOC(OCV(t),T(t)) = SOC_{0}[OCV(t)] + T(t)\times SOC_{rel}[OCV(t)]$$<br>

$$\begin{bmatrix}SOC|T_{1}\\SOC|T_{2}\\ ...\\ SOC|T_{3}\end{bmatrix} =\begin{bmatrix}1 & T_{1}\\ 1 &  T_{2}\\...& ... \\ 1 & T_{3}\end{bmatrix}\cdot\begin{bmatrix}SOC_{0}[OCV(t)]\\SOC_{rel}[OCV(t))] \end{bmatrix}$$<br>

$$Y = A\cdot X$$

$$X = A\Y$$

In [10]:
minV = 2.00
maxV = 3.75
v = np.arange(minV,maxV+0.01,0.01)

# Custom values of SOC
z = np.arange(0, 1.01, 0.01)
ocv_z = np.zeros([len(temp_list),len(z)])
soc = np.zeros([len(temp_list),len(v)])

Negative SOC values are taken as it might happen that we record the voltages lower than the lowest stored soc point. We are not including this effect in our model.

In [11]:
for i in range(len(temp_list)):

    OCV_0_z = interp_ocv(SOC,OCV_0)(z)
    OCV_rel_z = interp_ocv(SOC,OCV_rel)(z)

    ocv_z[i] = OCV_0_z + temp_list[i]*OCV_rel_z
    soc[i,:] = np.interp(v,ocv_z[i],z)
    

In [12]:
SOC_0 = np.zeros(len(v))
SOC_rel =np.zeros(len(v))

In [15]:
for i in range(len(v)):
    
    X = np.linalg.lstsq(A,soc[:,i], rcond = None)
    
    SOC_0[i] = X[0][0]
    SOC_rel[i] = X[0][1]

Thus if the voltage and temperature is known, we can predict the static value of the SOC by the formula

$$SOC(OCV(t),T(t)) = SOC_{0}[OCV(t)] + T(t)\times SOC_{rel}[OCV(t)]$$<br>



In [22]:
out_d = {}

out_d['OCV_0'] = OCV_0
out_d['OCV_rel'] = OCV_rel
out_d['SOC'] = SOC
out_d['v'] = v
out_d['SOC_0'] = SOC_0
out_d['SOC_rel'] = SOC_rel

__Exporting the Dictionary__

In [23]:
import pickle

f = open("Static Model.pkl","wb")

pickle.dump(out_d,f)

# close file
f.close()